<a href="https://colab.research.google.com/github/yy1200/COVID-19/blob/main/COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import pandas as pd
import numpy as np

#df = pd.read_csv("/content/covid-variants.csv",index_col='date',infer_datetime_format=True)
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

KeyboardInterrupt: ignored

In [ ]:
!kaggle datasets download -d 'yamqwe/omicron-covid19-variant-daily-cases'

In [ ]:
! mkdir omicron-covid19
! unzip omicron-covid19-variant-daily-cases.zip -d omicron-covid19

In [ ]:
df = pd.read_csv('omicron-covid19/covid-variants.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
Tnum=df.groupby('location')['num_sequences_total'].sum()

#NTnum=sorted(Tnum,reverse=True)
X=Tnum.keys()
Y=Tnum
plt.figure(figsize=(40,5))
plt.plot(X,Y)
plt.xticks(rotation=45)
plt.show()
#Tnum

In [ ]:
country={'City':list(Tnum.keys())}
data = pd.DataFrame(country)
data
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
   
longitude = []
latitude = []
def findGeocode(city):  
  try:
    geolocator = Nominatim(user_agent="your_app_name")
    return geolocator.geocode(city)
  except GeocoderTimedOut:
    return findGeocode(city)
for i in (country["City"]):
  if findGeocode != None:
    loc = findGeocode(i)
    latitude.append(loc.latitude)
    longitude.append(loc.longitude)
  else:
    latitude.append(np.nan)
    longitude.append(np.nan)

In [ ]:
country["Longitude"] = longitude
country["Latitude"] = latitude

country["Total numbers"]=Tnum
Ncountry=pd.DataFrame(country)
Ncountry

In [ ]:
t=Ncountry.sort_values(by=['Total numbers'],ascending=True)
t.head()

In [ ]:
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import folium_static
#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(Ncountry)):
  lat = Ncountry.iloc[i]['Latitude']
  long = Ncountry.iloc[i]['Longitude']
  radius=5
  popup_text = """Country : {}<br>
              %of Tnums : {}<br>"""
  popup_text = popup_text.format(Ncountry.iloc[i]['City'],Ncountry.iloc[i]['Total numbers'])
  folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

In [ ]:
import numpy as np
from folium.plugins import HeatMap

fmap = folium.Map()
Ncountry["Percetage"]=Ncountry["Total numbers"]/Ncountry["Total numbers"].sum()*1000000
a=Ncountry[["Latitude","Longitude","Percetage"]]
#print(a)
city_list=a.values.tolist()
fmap.add_child(HeatMap(data=city_list,overlay=False,show=False))
fmap

In [ ]:
circle_map = folium.Map(location=[30, 15], zoom_start=2, tiles='OpenStreetMap')
for i in range(len(Ncountry)):
  lat = Ncountry.iloc[i]['Latitude']
  long = Ncountry.iloc[i]['Longitude']
  popup_text = """Country : {}<br>
            %of Tnums : {}<br>"""
  popup_text = popup_text.format(Ncountry.iloc[i]['City'],Ncountry.iloc[i]['Total numbers'])
  radius = Ncountry.iloc[i]["Total numbers"]/15
  folium.Circle(location = [lat, long],radius=radius,popup=popup_text,max_width='100',fill=True).add_to(circle_map)
circle_map

In [ ]:
x=t.iloc[-10:,0]
y=t.iloc[-10:,3]
plt.xlabel("Number of Case")
plt.ylabel("Location")
plt.barh(x,y)
plt.show()

In [ ]:
for virus in df.variant.unique():
  df1 = df.loc[df['variant'] == virus].groupby('location')['num_sequences'].agg('sum').sort_values(ascending=True)[-10:]
  #dataframe = pd.DataFrame({'Location':dataframe.index, 'Number of Case':dataframe.values})
  X=df1.keys()
  Y=df1[X]
  plt.barh(X,Y)
  plt.xlabel('Number of Case')
  plt.ylabel('Location')
  plt.title('{} CASES'.format(virus),loc='center',fontweight="bold")
  plt.show()

In [ ]:
sample = df.loc[df['variant'] == 'Omicron'].groupby('date')['num_sequences'].agg('sum')
xo=sample.keys()
yo=sample[xo]
plt.figure(figsize=(15,5))
plt.plot(xo,yo)
plt.xticks(rotation=45)
plt.show()

In [ ]:
'''import requests
import io
url='https://covid-19.nchc.org.tw/api/covid19?CK=covid-19@nchc.org.tw&querydata=4002'
df=pd.read_csv(url)
for i in df:
  print(i)'''

Covid Prediction

In [ ]:
import csv
import pandas as pd
import numpy as np
df = pd.read_csv('omicron-covid19/covid-variants.csv')
df.set_index('date', inplace=True)
df

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt

class COVID19_Forecasts:
  def __init__(self,length=80,batch_size=1):
    self.length = length
    self.batch_size = batch_size
    
  def model(self,length,batch_size,s_train,s_test):
    model = Sequential()
    model.add(LSTM(100, input_shape=(length,s_train.shape[1])))
    #model.add(LSTM(256, return_sequences=True))
    #model.add(TimeDistributed(Dense(1)))
    #model.add(Flatten())
    #model.add(Dense(5,activation='linear'))
    model.add(Dense(s_train.shape[1]))
    #model.compile(loss="mean_absolute_error", optimizer="adam",metrics=['mean_absolute_error'])
    model.compile(optimizer='rmsprop', loss='mean_squared_error')
    model.summary()
    early_stop = EarlyStopping(monitor='val_loss',patience=25)
    train_data_gen = TimeseriesGenerator(s_train, s_train, length=length, batch_size=batch_size, sampling_rate=1,stride=1) 
    test_data_gen = TimeseriesGenerator(s_test, s_test, length=length, batch_size=batch_size, sampling_rate=1,stride=1)
    history=model.fit(train_data_gen,epochs=100,validation_data=test_data_gen,callbacks=[early_stop]).history
    
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.show()
    return model,train_data_gen,test_data_gen

  def data_preb(self,data,index_size):
    data.sort_index(ascending=True,inplace=True)
    train = data.iloc[:index_size]
    test = data.iloc[index_size:]
    scaler = MinMaxScaler()
    scaler.fit(train)
    s_train = scaler.transform(train)
    s_test = scaler.transform(test)
    return scaler,s_train,s_test,train,test

In [ ]:
covid = COVID19_Forecasts()

In [ ]:
df['Time']=df.index
data=pd.DataFrame(df.groupby('Time')['num_sequences_total'].sum())
#data=data.drop(['location','variant','perc_sequences','num_sequences'],axis=1,inplace=True)
scaler,s_train,s_test,train,test = covid.data_preb(data,40)

In [ ]:
model,train_data_gen,test_data_gen = covid.model(length=4,batch_size=1,s_train=s_train,s_test=s_test)

In [ ]:
score = model.evaluate(test_data_gen) 
trainPredict = model.predict(train_data_gen) 
testPredict = model.predict(test_data_gen)
print(score)

In [ ]:
pred= scaler.inverse_transform(trainPredict)
Y=data.index
plt.plot(Y[-5:],pred[-5:],label="Predictions")
plt.plot(Y[-5:],data.values[-5:],label="True")
plt.legend()
plt.show()